In [ ]:
import time
import numpy as np
import osmnx as ox
import pandas as pd
import networkx as nx 
import matplotlib.pyplot as plt
from itertools import combinations
from IPython.display import clear_output
import random
import matplotlib.cm as cm
import matplotlib.colors as colors
from IPython.display import Image
from pprint import pprint
%matplotlib inline

In [ ]:
def get_seeds(G_bike, G_drive, pairs):
    """Get x pairs of random pairs of nodes in the bike and street layer.

    Parameters
    ----------
    G_bike : type
        Description of parameter `G_bike`.
    G_drive : type
        Description of parameter `G_drive`.
    pairs : type
        Description of parameter `pairs`.

    Returns
    -------
    type
        Description of returned object.

    """
    seeds_bike = []
    seeds_car = []
    u = 0
    for u in range(pairs):
        i = random.choice(list(G_bike.nodes(data=True)))
        j = random.choice(list(G_bike.nodes(data=True)))
        u = ox.get_nearest_node(G_drive, (i[1]['y'], i[1]['x']))
        v = ox.get_nearest_node(G_drive, (j[1]['y'], j[1]['x']))
        if u != v or i[0] != j[0]:
            seeds_car.append((u, v))
            seeds_bike.append((i[0], j[0]))
        else:
            i = random.choice(list(G_bike.nodes(data=True)))
            j = random.choice(list(G_bike.nodes(data=True)))
            u = ox.get_nearest_node(G_drive, (i[1]['y'], i[1]['x']))
            v = ox.get_nearest_node(G_drive, (j[1]['y'], j[1]['x']))
    return seeds_bike, seeds_car

In [ ]:
#euclidean distance
def euclidean_dist_vec(y1, x1, y2, x2):
    '''
    Calculate the euclidean distance between two points.
    '''
    
    #distance = ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5  #use this one if the graph is projected
    distance = ox.distance.great_circle_vec(y1, x1, y2, x2)  #use this one if the graph is not projected
    #print('euclidean distance=',distance)
    return distance

In [ ]:
#shortest path
def get_travel_distance(G, u_v):
    """Find the shortest path between two nodes and calculate the travel distance.

    Parameters
    ----------
    G : nx.Graph
        Networkx graph.
    u_v : touple
        Touple containing the origin and destination for the path.

    Returns
    -------
    int
        Distance for the path between 'u' to 'v'.

    """
    path = nx.shortest_path(G, u_v[0], u_v[1], weight='length')
    distance = 0
    for i, j in zip(path[:-1], path[1:]):
        #distance += float(G[i][j][0]['length'])
        try:
            distance += float(G[i][j][0]['length'])
        except:
            distance += euclidean_dist_vec(G.nodes[i]['y'], G.nodes[i]['x'], G.nodes[j]['y'], G.nodes[j]['x'])
            print('Error: {} {}, distance: {}'.format(G, u_v,euclidean_dist_vec(G.nodes[i]['y'],G.nodes[i]['x'], G.nodes[j]['y'], G.nodes[j]['x'])))
    #print('travel distance=',distance)
    return distance

In [ ]:

def calculate_directness(G_bike, G_drive, name, seeds_bike, car_value):
    d_ij_b = []
    d_ij_s = []
    print('Calculating {}'.format(name))
    try:
        G_bike.add_edge(int(row['i']), int(row['j']), length=euclidean_dist_vec(G_bike.nodes[row['i']]['y'],G_bike.nodes[row['i']]['x'], G_bike.nodes[row['j']]['y'], G_bike.nodes[row['j']]['x']))
    except:
        pass
        
        avg_bike = [] 
        for i_j in seeds_bike:
            if nx.has_path(G_bike, i_j[0], i_j[1]):
                euclidean_distance = euclidean_dist_vec(G_bike.nodes[i_j[0]]['y'], G_bike.nodes[i_j[0]]['x'], G_bike.nodes[i_j[1]]['y'], G_bike.nodes[i_j[0]]['x'])
                bike_distance = get_travel_distance(G_bike, i_j)
                #avg_bike.append(euclidean_distance)
                avg_bike.append(euclidean_distance/bike_distance)
            else:
                avg_bike.append(0)
        bike_value = np.average(avg_bike)
        d_ij_b.append(bike_value)
        d_ij_s.append(car_value)
        #print('bike_distance=',bike_distance)
        print('{} Efficiency: {}/{}'.format(name, round(bike_value, 3), round(car_value, 3)))
        
    #print('Efficiency: {}/{}'.format(round(bike_value, 3), round(car_value, 3)))
    return d_ij_b, G_bike




In [ ]:
# network getting

# get 2021-04-01 bike networks(.graphml file and .png file) of Milano


ox.utils.config(overpass_settings='[out:json][timeout:180][date:"2021-04-01T00:00:00Z"]')

place = 'Milano, Italy'

cf1 = '["highway"~"cycleway"]["bicycle"!~"no"]'
cf2 = '["cycleway"]["cycleway"!~"no"]'
cf3 = '["cycleway:right"]["cycleway:right"!~"no"]'
cf4 = '["cycleway:left"]["cycleway:left"!~"no"]'
cf5 = '["cycleway:both"]["cycleway:both"!~"no"]'
cf6 = '["bicycle"~"designated"]'
cf7 = '["oneway:bicycle"]'




G_21_b1=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf1)
G_21_b2=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf2)
G_21_b3=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf3)
G_21_b4=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf4)
G_21_b5=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf5)
G_21_b6=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf6)
G_21_b7=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf7)

G_21_b11 = nx.compose(G_21_b1,  G_21_b2)
G_21_b12 = nx.compose(G_21_b11, G_21_b3)
G_21_b13 = nx.compose(G_21_b12, G_21_b4)
G_21_b14 = nx.compose(G_21_b13, G_21_b5)
G_21_b15 = nx.compose(G_21_b14, G_21_b6)
G_21_b16 = nx.compose(G_21_b15, G_21_b7)




G_21_b44=ox.simplification.consolidate_intersections(ox.project_graph(G_21_b16,to_crs='EPSG:3812'), tolerance=15, rebuild_graph=True, dead_ends=True, reconnect_edges=True)
G_21_b5=ox.project_graph(G_21_b44, to_crs='EPSG:4326')
fig, ax1_21 = ox.plot_graph(G_21_b5,node_size=0,bgcolor='white',edge_color='black',edge_linewidth=0.5,save=False,filepath='data_indicator/Milano/Milano_b21.png',dpi=1080)
#ox.io.save_graphml(G_21_b5, filepath='data_indicator/Milano/Milano_21_bike.graphml', gephi=False, encoding='utf-8')
print('Milano 21_bike_net saved')


In [ ]:
# get 2021-04-01 car networks(.graphml file) of Milano

ox.utils.config(overpass_settings='[out:json][timeout:180][date:"2021-04-01T00:00:00Z"]')

place = 'Milano, Italy'

G_21_d1=ox.graph.graph_from_place(place, network_type='drive', simplify=False, retain_all=False, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter=None)

G_21_d2=ox.simplification.consolidate_intersections(ox.project_graph(G_21_d1,to_crs='EPSG:3812'), tolerance=15, rebuild_graph=True, dead_ends=False, reconnect_edges=True)
G_21_d3=ox.project_graph(G_21_d2, to_crs='EPSG:4326')


#ox.io.save_graphml(G_21_d3, filepath='data_indicator/Milano/Milano_21_drive.graphml', gephi=False, encoding='utf-8')
print('Milano 21_drive_net saved')


In [ ]:
# network getting

# get 2020-01-01 bike networks(.graphml file and .png file) of Milano


ox.utils.config(overpass_settings='[out:json][timeout:180][date:"2020-01-01T00:00:00Z"]')

place = 'Milano, Italy'

cf1 = '["highway"~"cycleway"]["bicycle"!~"no"]'
cf2 = '["cycleway"]["cycleway"!~"no"]'
cf3 = '["cycleway:right"]["cycleway:right"!~"no"]'
cf4 = '["cycleway:left"]["cycleway:left"!~"no"]'
cf5 = '["cycleway:both"]["cycleway:both"!~"no"]'
cf6 = '["bicycle"~"designated"]'
cf7 = '["oneway:bicycle"]'



G_20_b1=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf1)
G_20_b2=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf2)
G_20_b3=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf3)
G_20_b4=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf4)
G_20_b5=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf5)
G_20_b6=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf6)
G_20_b7=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf7)

G_20_b11 = nx.compose(G_20_b1,  G_20_b2)
G_20_b12 = nx.compose(G_20_b11, G_20_b3)
G_20_b13 = nx.compose(G_20_b12, G_20_b4)
G_20_b14 = nx.compose(G_20_b13, G_20_b5)
G_20_b15 = nx.compose(G_20_b14, G_20_b6)
G_20_b16 = nx.compose(G_20_b15, G_20_b7)


G_20_b44=ox.simplification.consolidate_intersections(ox.project_graph(G_20_b16,to_crs='EPSG:3812'), tolerance=15, rebuild_graph=True, dead_ends=True, reconnect_edges=True)
G_20_b5=ox.project_graph(G_20_b44, to_crs='EPSG:4326')
fig, ax1_20 = ox.plot_graph(G_20_b5,node_size=0,bgcolor='white',edge_color='black',edge_linewidth=0.5,save=False,filepath='data_indicator/Milano/Milano_b20.png',dpi=1080)
#ox.io.save_graphml(G_20_b5, filepath='data_indicator/Milano/Milano_20_bike.graphml', gephi=False, encoding='utf-8')
print('Milano 20_bike_net saved')




In [ ]:
# get 2020-01-01 car networks(.graphml file) of Milano

ox.utils.config(overpass_settings='[out:json][timeout:180][date:"2020-01-01T00:00:00Z"]')

place = 'Milano, Italy'

G_20_d1=ox.graph.graph_from_place(place, network_type='drive', simplify=False, retain_all=False, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter=None)

G_20_d2=ox.simplification.consolidate_intersections(ox.project_graph(G_20_d1,to_crs='EPSG:3812'), tolerance=15, rebuild_graph=True, dead_ends=False, reconnect_edges=True)
G_20_d3=ox.project_graph(G_20_d2, to_crs='EPSG:4326')


#ox.io.save_graphml(G_20_d3, filepath='data_indicator/Milano/Milano_20_drive.graphml', gephi=False, encoding='utf-8')
print('Milano 20_drive_net saved')


In [ ]:
print(nx.number_connected_components(ox.utils_graph.get_undirected(G_21_b16)))
print(nx.number_connected_components(ox.utils_graph.get_undirected(G_20_b16)))


In [ ]:

# get the edges added by covering G_20 on G_21, the remained red ones are edges really added 
m = ox.plot_graph_folium(G_21_b16, weight=1, color='red')
m = ox.plot_graph_folium(G_20_b16,graph_map=m,weight=1,color='black')

m.save('data_indicator/Milano/Milano_edge_add.html')

In [ ]:

# get the edges deleted by covering G_21 on G_20, the remained red ones are edges really added 
m = ox.plot_graph_folium(G_20_b16, weight=1, color='blue')
m = ox.plot_graph_folium(G_21_b16,graph_map=m,weight=1.5,color='black')

m.save('data_indicator/Milano/Milano_edge_del.html')

In [ ]:
# Getting top n Largest Connected Components

G = G_21_b5

#ox.plot_graph(G,node_size=0,edge_linewidth=0.5,edge_color='w')
G_bike = G.copy()
m = ox.plot_graph_folium(G_bike, weight=1, color='white')
    
    
n=10
i=0
    

    
WCC_color = ox.plot.get_colors(n, cmap='rainbow', start=0.0, stop=0.7, alpha=1.0, return_hex=True)
#print(WCC_color)
    
    
while i<=n-1:
    CC=ox.utils_graph.get_largest_component(G_bike, strongly=False)
    #ox.plot_graph(CC,node_size=0,edge_linewidth=0.5,edge_color='w')
    G_bike.remove_edges_from(CC.edges)
    
    m = ox.plot_graph_folium(CC, graph_map=m, weight=1, color=WCC_color[i])
    #print('m{} completed'.format(i+1))
    i+=1
m = ox.plot_graph_folium(G_bike, graph_map=m, weight=1, color='black')

m.save('data_indicator/Milano/Milano_21_WCCs.html')
print('Milano 21 nw_CC saved')
    



In [ ]:
# Getting top n Largest Connected Components

G = G_20_b5

#ox.plot_graph(G,node_size=0,edge_linewidth=0.5,edge_color='w')
G_bike = G.copy()
m = ox.plot_graph_folium(G_bike, weight=1, color='white')
    
    
n=10
i=0
    

    
WCC_color = ox.plot.get_colors(n, cmap='rainbow', start=0.0, stop=0.7, alpha=1.0, return_hex=True)
#print(WCC_color)
    
    
while i<=n-1:
    CC=ox.utils_graph.get_largest_component(G_bike, strongly=False)
    #ox.plot_graph(CC,node_size=0,edge_linewidth=0.5,edge_color='w')
    G_bike.remove_edges_from(CC.edges)
    
    m = ox.plot_graph_folium(CC, graph_map=m, weight=1, color=WCC_color[i])
    #print('m{} completed'.format(i+1))
    i+=1
m = ox.plot_graph_folium(G_bike, graph_map=m, weight=1, color='black')

m.save('data_indicator/Milano/Milano_20_WCCs.html')
print('Milano 20 nw_CC saved')
    



In [ ]:
# Getting top n Largest Connected Components

G = G_21_b5

#ox.plot_graph(G,node_size=0,edge_linewidth=0.5,edge_color='w')
G_bike = G.copy()
m = ox.plot_graph_folium(G_bike, weight=1, color='red',opacity=0.7)
    

G = G_20_b5

#ox.plot_graph(G,node_size=0,edge_linewidth=0.5,edge_color='w')
G_bike = G.copy()
    
n=10
i=0
    
WCC_color = ox.plot.get_colors(n, cmap='rainbow', start=0.0, stop=0.7, alpha=1.0, return_hex=True)
#print(WCC_color)
    
while i<=n-1:
    CC=ox.utils_graph.get_largest_component(G_bike, strongly=False)
    
    G_bike.remove_edges_from(CC.edges)
    
    m = ox.plot_graph_folium(CC, graph_map=m, weight=1.5, color=WCC_color[i])
    i+=1
m = ox.plot_graph_folium(G_bike, graph_map=m, weight=1.5, color='black')


    
m.save('data_indicator/Milano/Milano_test1.html')

    



In [ ]:
#calculate the network efficiency(connectedness) of networks (table 5.1)

g = ox.utils_graph.get_digraph(G_21_21, weight='length')
g_21 = g.to_undirected()

n = len(g_21)
denom = n * (n - 1)
if denom != 0:
    lengths = nx.all_pairs_dijkstra_path_length(g_21, weight="weight")
    g_eff = 0
    for source, targets in lengths:
        for target, distance in targets.items():
            if distance > 0:
                g_eff += (1 / distance)
    g_eff /= denom
    # g_eff = sum(1 / d for s, tgts in lengths
    #                   for t, d in tgts.items() if d > 0) / denom
else:
    g_eff = 0
    
print('eff of 21 Milano {}'.format(g_eff))

g = ox.utils_graph.get_digraph(G_20_21, weight='length')
g_20 = g.to_undirected()

n = len(g_20)
denom = n * (n - 1)
if denom != 0:
    lengths = nx.all_pairs_dijkstra_path_length(g_20, weight="weight")
    g_eff = 0
    for source, targets in lengths:
        for target, distance in targets.items():
            if distance > 0:
                g_eff +=( 1 / distance)
    g_eff /= denom
    # g_eff = sum(1 / d for s, tgts in lengths
    #                   for t, d in tgts.items() if d > 0) / denom
else:
    g_eff = 0
    
print('eff of 20 Milano {}'.format(g_eff))

In [ ]:
for i in range(1,6):

    # calculate the directness and connectedness of 2020 bike networks of Milano
    '''
    Right now there is a bug about loading .graphml file which is saved from intersections consolidated graph, 
    so the graph has to be used at once. But the bug was recently fixed and will appear in the next release, 
    targeted for early May 2021, just right after the paper written. So maybe graphs can be gotten by: 

    G_bike = ox.load_graphml('data_indicator/Milano/Milano_20_bike.graphml')


    '''
    print('calculating indicators of 2020 bike')

    seeds = 1000# how many node pairs are going to be chosen

    G_drive = G_20_d1
    G_drive = ox.get_undirected(G_drive)

    avg_street = []
    avg_street_1=[]
    avg_street_2=[]
    G_bike = G_20_b16
    G_bike = ox.get_undirected(G_bike)   
    seeds_bike, seeds_car = get_seeds(G_bike, G_drive, seeds) 
    for u_v in seeds_car:
        euclidean_distance = euclidean_dist_vec(G_drive.nodes[u_v[0]]['y'],G_drive.nodes[u_v[0]]['x'], G_drive.nodes[u_v[1]]['y'], G_drive.nodes[u_v[0]]['x'])
        travel_distance = get_travel_distance(G_drive, u_v)
        if travel_distance==0:
            euclidean_distance=0.0
            travel_distance=1.0
        else:
            result=travel_distance
            avg_street.append(euclidean_distance/travel_distance)


    car_value = np.average(avg_street)  # Average efficiency in the car layer
    d_ij_b, G_bike = calculate_directness(G_bike, G_drive, 'Milano', seeds_bike, car_value)




In [ ]:
for i in range(1,4):

    # calculate the directness and connectedness of 2021 bike networks of Milano
    '''
    Right now there is a bug about loading .graphml file which is saved from intersections consolidated graph, 
    so the graph has to be used at once. But the bug was recently fixed and will appear in the next release, 
    targeted for early May 2021, just right after the paper written. So maybe graphs can be gotten by: 

    G_bike = ox.load_graphml('data_indicator/Milano/Milano_21_bike.graphml')


    '''
    print('calculating indicators of 2021 bike')

    seeds = 1000# how many node pairs are going to be chosen

    G_drive = G_21_d1
    G_drive = ox.get_undirected(G_drive)

    avg_street = []
    avg_street_1=[]
    avg_street_2=[]
    G_bike = G_21_b16
    G_bike = ox.get_undirected(G_bike)   
    seeds_bike, seeds_car = get_seeds(G_bike, G_drive, seeds) 
    for u_v in seeds_car:
        euclidean_distance = euclidean_dist_vec(G_drive.nodes[u_v[0]]['y'],G_drive.nodes[u_v[0]]['x'], G_drive.nodes[u_v[1]]['y'], G_drive.nodes[u_v[0]]['x'])
        travel_distance = get_travel_distance(G_drive, u_v)
        if travel_distance==0:
            euclidean_distance=0.0
            travel_distance=1.0
        else:
            result=travel_distance
            avg_street.append(euclidean_distance/travel_distance)


    car_value = np.average(avg_street)  # Average efficiency in the car layer
    d_ij_b, G_bike = calculate_directness(G_bike, G_drive, 'Milano', seeds_bike, car_value)




In [ ]:
# getting the basic statistical data of 20_network (table 3.1)

d_original = ox.stats.basic_stats(G_20_b16)
print('Milano 20_d_original:\n{}'.format(d_original))


In [ ]:
# getting the basic statistical data of 21_network (table 3.1)

d_original = ox.stats.basic_stats(G_21_b16)
print('Milano 21_d_original:\n{}'.format(d_original))
